## Stuff to load everything

In [4]:
import rootutils

rootutils.setup_root("/TopoBenchmarkX", indicator=".project-root", pythonpath=True)

import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [5]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader

data_loader = HypergraphLoader(config)
data = data_loader.load()

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072


In [6]:
print(data)

Data(x=[2708, 1433], edge_index=[2708, 4585], y=[2708], n_x=2708, num_hyperedges=1072, num_class=7)


In [7]:
import torch_geometric
import torch

In [8]:
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.utils import to_undirected

In [9]:
edges = data['edge_index'].coalesce().indices()
edges = to_undirected(edges)

In [10]:
subset, _, _, _ = k_hop_subgraph(2, 1, edges)
print(subset)

tensor([   2, 3107, 3655])


In [56]:
import copy

class KHopLifting(torch_geometric.transforms.BaseTransform):
    def __init__(self, k=1):
        super().__init__()
        self.k = k
        
    def forward(self, data: torch_geometric.data.Data, slices: dict = None) -> dict:
        data_lifted = copy.copy(data)
        
        n_nodes = data.x.shape[0]
        
        if slices is not None:
            n_edges = data.edge_index.shape[1]
            fix_idxs = torch.zeros((2,n_edges), dtype=slices["edge_index"].dtype)
            #slices["edge_index"][-1] -= 1
            last_idx = 0
            for i, idx in enumerate(slices["edge_index"]):
                if not i==0:
                    fix_idxs[:,last_idx:idx] = slices["x"][i-1]
                    last_idx = idx
            data_lifted.edge_index += fix_idxs
            
        incidence_1 = torch.zeros(n_nodes, n_nodes)
        edge_index = torch_geometric.utils.to_undirected(data.edge_index)
        for n in range(n_nodes):
            neighbors, _, _, _ = k_hop_subgraph(n, self.k, edge_index)
            incidence_1[n, neighbors] = 1
        incidence_1 = torch.Tensor(incidence_1).to_sparse_coo()
        data_lifted["hyperedges"] = incidence_1
        return data_lifted
    
    def __call__(self, data, slices = None):
        return self.forward(data, slices)

In [57]:
data_mutag = torch_geometric.datasets.TUDataset(root="../data/graph",name="MUTAG")
data = data_mutag._data
slices = data_mutag.slices
print(slices)

{'edge_index': tensor([   0,   38,   66,   94,  138,  160,  222,  256,  300,  326,  364,  402,
         448,  498,  526,  570,  620,  642,  680,  708,  748,  786,  824,  878,
         944,  982, 1008, 1062, 1100, 1150, 1204, 1254, 1304, 1354, 1382, 1420,
        1450, 1486, 1518, 1550, 1578, 1616, 1644, 1688, 1726, 1750, 1804, 1854,
        1892, 1936, 1970, 2030, 2090, 2132, 2174, 2204, 2242, 2286, 2344, 2398,
        2438, 2476, 2498, 2546, 2590, 2624, 2658, 2704, 2758, 2800, 2830, 2890,
        2926, 2960, 3014, 3052, 3072, 3106, 3140, 3178, 3222, 3246, 3302, 3338,
        3360, 3414, 3468, 3504, 3528, 3556, 3610, 3668, 3712, 3766, 3810, 3850,
        3906, 3946, 3974, 4008, 4042, 4096, 4152, 4196, 4250, 4288, 4334, 4390,
        4434, 4500, 4550, 4572, 4606, 4632, 4666, 4692, 4712, 4756, 4810, 4854,
        4882, 4938, 4986, 5022, 5044, 5088, 5132, 5176, 5210, 5238, 5260, 5300,
        5322, 5368, 5418, 5440, 5486, 5536, 5564, 5586, 5620, 5642, 5686, 5710,
        5732, 5766, 5800,

In [58]:
lift = KHopLifting(k=1)

data_lifted = lift(data, slices)

print(data_mutag)
print(data)
print(data_lifted)

MUTAG(188)
Data(x=[3371, 7], edge_index=[2, 7442], edge_attr=[7442, 4], y=[188])
Data(x=[3371, 7], edge_index=[2, 7442], edge_attr=[7442, 4], y=[188], hyperedges=[3371, 3371])


In [62]:
print(data_lifted.hyperedges.indices()[1][data_lifted.hyperedges.indices()[0]==0])

tensor([0, 1, 5])


In [20]:
def get_cora(cfg):
    data_dir = cfg["data_dir"]
    print(f"Loading {cfg['data_domain']} dataset name: {cfg['data_name']}")
    
    data = torch_geometric.datasets.Planetoid(data_dir, "Cora")
    return data

In [21]:
from torch_geometric.loader.dataloader import DataLoader

data_cora = get_cora(config.data)

Loading hypergraph dataset name: cora


In [22]:
data = data_cora._data
slices = data_cora.slices
print(slices)
data_lifted = lift(data)

print(data_cora)
print(data)
print(data_lifted)

None
Cora()
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], hyperedges=[2708, 2708])


In [28]:
print(data_lifted.hyperedges.indices()[1][data_lifted.hyperedges.indices()[0]==2])

tensor([   1,    2,  332, 1454, 1666, 1986])


In [14]:
model = hydra.utils.instantiate(config.model)

/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout_workaround' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout_workaround'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout'])`.


In [ ]:
results = model(data_lifted.x, data_lifted.hyperedges)